In [ ]:
# importing modules I will use
import pandas as pd
import json
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
from itertools import combinations
import os
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import ast
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from gensim.models import KeyedVectors
import string
from nltk.tokenize import RegexpTokenizer
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
device = torch.device('cuda:0') #if torch.cuda.is_available() else 'cpu')
from PIL import Image
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score


## MODEL

In [ ]:
df = pd.read_csv('../data/df_subm.csv')
# The split to val, test and train has taken place in the ai_project_preprocessing.ipynb
val_df = pd.read_csv('../data/val_subm.csv')
test_df = pd.read_csv('../data/test_subm.csv')
train_df = pd.read_csv('../data/train_subm.csv')

In [ ]:
# I have the tensors saved locally, I can upload them on my repo too
# I usually start things from this point

val_tensors = torch.load('../data/val_subm.pt')
test_tensors = torch.load('../data/test_subm.pt')
train_tensors = torch.load('../data/train_subm.pt')

In [ ]:
df['genres']=df['genres'].apply(lambda x: ast.literal_eval(x))

train_df['genres']=train_df['genres'].apply(lambda x: ast.literal_eval(x))
test_df['genres']=test_df['genres'].apply(lambda x: ast.literal_eval(x))
val_df['genres']=val_df['genres'].apply(lambda x: ast.literal_eval(x))


In [ ]:
train_df['production_companies']=train_df['production_companies'].apply(lambda x: ast.literal_eval(x))
test_df['production_companies']=test_df['production_companies'].apply(lambda x: ast.literal_eval(x))
val_df['production_companies']=val_df['production_companies'].apply(lambda x: ast.literal_eval(x))



In [ ]:
train_df['production_companies']=train_df['production_companies'].apply(lambda x: (' ').join(x))
test_df['production_companies']=test_df['production_companies'].apply(lambda x: (' ').join(x))
val_df['production_companies']=val_df['production_companies'].apply(lambda x: (' ').join(x))

In [ ]:
mlb = MultiLabelBinarizer()
# We need to pass a list of lists of the labels to get a one-hot encoding for the train, the test and the validation dataset.

# Fitting the multilabel binarizer to the labels available.
train_labels = mlb.fit_transform(train_df['genres'].tolist())
test_labels = mlb.fit_transform(test_df['genres'].tolist())
val_labels = mlb.fit_transform(val_df['genres'].tolist())

In [ ]:
# This function creates a dictionary that has the number of movie labels as a key 
# and the number of occurrences of this number of labels as a value.

def plot_num_labels_movies(genre_lists):
    dic = {}
    for genre_list in genre_lists:
        if len(genre_list) in dic:
            dic[len(genre_list)] += 1
            
        else:
            dic[len(genre_list)] = 1
    return dic 

plot_dic = plot_num_labels_movies(df.genres.tolist())
plt.bar(*zip(*plot_dic.items()))
plt.title('Occurrences of different number of labels in the dataset')
plt.xlabel('Number of labels')
plt.ylabel('Number of occurrences')
plt.show()


In [ ]:
swords = stopwords.words('english')
reg_ex_tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

def clean(text,is_for_summary=True):
    # removing punctuation and digits
    text = ''.join([c for c in text if c not in string.punctuation and not c.isdigit()])
    # tokenizing with the RegexpTokenizer and lowercasing the text
    text = reg_ex_tokenizer.tokenize(text.lower())
    # removing stop words if cleaning is applied on the overview - if we clean up the titles we keep the stop words since certain titles contain solely stop words(e.g. 'Who am I?').
    # maybe I should filter out stop words in titles too and only keep them when there are no other words.
    if is_for_summary:        
        text = [w for w in text if w not in swords]
    # lemmatizing the words with the WordNetLemmatizer and using join to put them back into a string
    text = ' '.join([lemmatizer.lemmatize(w) for w in text])
    return text

In [ ]:
# Cleaning up the overview column.
train_df['overview'] = train_df['overview'].apply(lambda x: clean(x))
test_df['overview'] = test_df['overview'].apply(lambda x: clean(x))
val_df['overview'] = val_df['overview'].apply(lambda x: clean(x))

# Cleaning up the original title column.
train_df['original_title'] = train_df['original_title'].apply(lambda x: clean(x,False))
test_df['original_title'] = test_df['original_title'].apply(lambda x: clean(x,False))
val_df['original_title'] = val_df['original_title'].apply(lambda x: clean(x,False))

# Cleaning up the production companies column.
train_df['production_companies'] = train_df['production_companies'].apply(lambda x: clean(x))
test_df['production_companies'] = test_df['production_companies'].apply(lambda x: clean(x))
val_df['production_companies'] = val_df['production_companies'].apply(lambda x: clean(x))


In [ ]:
# Merging the three columns containing textual data into one to cover all possible occurrences for words.
df_tit_overview = df[['original_title','overview','production_companies']] 
df_vocab=df_tit_overview.stack().reset_index()
# Getting length of sequences for padding
max_len_for_padding = df_vocab[0].map(len).max()
embedding_dim = 300
tokenizer = Tokenizer(num_words=50000, lower=True)
tokenizer.fit_on_texts(df_vocab[0].values)
# Creating a word to integer dictionary for my vocab.
word_index = tokenizer.word_index


In [ ]:
def embed_vocab(pretrained_dict, vocab, emb_size):
    # creating a dictionary with keys the words and values the pretrained word vectors. 
    embeddings_index = {}
    for w in pretrained_dict.wv.vocab:
        embeddings_index[w] = pretrained_dict.word_vec(w)     
    # instantiating matrix with shape (vocab + 1, 300)
    embedding_matrix = 1 * np.random.randn(len(vocab)+1, emb_size)
    # looking up the words in my vocab
    for word, i in vocab.items():
        i-=1
        # getting the pretrained vector for the corresponding word
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # adding the vector to the matrix
            embedding_matrix[i] = embedding_vector                      
    del(embeddings_index)
        
    return embedding_matrix

# need to download the pretrained vectors file. uncomment to download and unzip
#!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
#!gunzip /content/GoogleNews-vectors-negative300.bin.gz

w2v_dic = KeyedVectors.load_word2vec_format("/home/gusmavko@GU.GU.SE/aics-project/data/GoogleNews-vectors-negative300.bin", binary=True)
word2vec_matrix = embed_vocab(w2v_dic, word_index, 300)

    

In [ ]:
# Transforming each word token from the textual data into the corresponding index. 
# TODO maybe when padding do I need to pad the titles to the same length as the overview 

X_overview_train = tokenizer.texts_to_sequences(train_df['overview'].values)
X_overview_train = pad_sequences(X_overview_train, maxlen=max_len_for_padding)

X_overview_test = tokenizer.texts_to_sequences(test_df['overview'].values)
X_overview_test = pad_sequences(X_overview_test, maxlen=max_len_for_padding)

X_overview_val = tokenizer.texts_to_sequences(val_df['overview'].values)
X_overview_val = pad_sequences(X_overview_val, maxlen=max_len_for_padding)

print(f'Train overview shape:{X_overview_train.shape}, Test overview shape:{X_overview_test.shape}, Validation overview shape:{X_overview_val.shape}')

In [ ]:
# Tranforming the titles to integer representations.
max_len_title = train_df['original_title'].map(len).max()
# train title

X_title_train = tokenizer.texts_to_sequences(train_df['original_title'].values)
X_title_train = pad_sequences(X_title_train,maxlen=max_len_title)
# test title

X_title_test = tokenizer.texts_to_sequences(test_df['original_title'].values)
X_title_test = pad_sequences(X_title_test,maxlen=max_len_title)

# val title
X_title_val = tokenizer.texts_to_sequences(val_df['original_title'].values)
X_title_val = pad_sequences(X_title_val,maxlen=max_len_title)

print(f'Train title shape:{X_title_train.shape}, Test overview shape:{X_title_test.shape}, Validation overview shape:{X_title_val.shape}')

In [ ]:
# transforming the companies to integer representations
max_len_comp = train_df['production_companies'].map(len).max()
X_company_train = tokenizer.texts_to_sequences(train_df['production_companies'].values)
X_company_train = pad_sequences(X_company_train, maxlen=max_len_comp)

X_company_test = tokenizer.texts_to_sequences(test_df['production_companies'].values)
X_company_test = pad_sequences(X_company_test, maxlen=max_len_comp)

X_company_val = tokenizer.texts_to_sequences(val_df['production_companies'].values)
X_company_val = pad_sequences(X_company_val, maxlen=max_len_comp)


In [ ]:
# Changing the dimensions of the tensors to later pass them in batches to the model.
X_img_val = val_tensors.permute(0,3,1,2)

X_img_test = test_tensors.permute(0,3,1,2)

X_img_train = train_tensors.permute(0,3,1,2)

In [ ]:
batch_size = 150
img_train_data = TensorDataset(X_img_train, torch.from_numpy(train_labels))
img_val_data = TensorDataset(X_img_val, torch.from_numpy(val_labels))
img_test_data = TensorDataset(X_img_test, torch.from_numpy(test_labels))

# Dataloaders for the images
img_train_loader = DataLoader(img_train_data, batch_size=batch_size)
img_val_loader = DataLoader(img_val_data, batch_size=batch_size)
img_test_loader = DataLoader(img_test_data, batch_size=batch_size)

In [ ]:
# using TensorDataset to transform the arrays into tensors for my overview title data

title_train_data = TensorDataset(torch.from_numpy(X_title_train), torch.from_numpy(train_labels))
title_val_data = TensorDataset(torch.from_numpy(X_title_val), torch.from_numpy(val_labels))
title_test_data = TensorDataset(torch.from_numpy(X_title_test), torch.from_numpy(test_labels))

title_train_loader = DataLoader(title_train_data, batch_size=batch_size)
title_val_loader = DataLoader(title_val_data, batch_size=batch_size)
title_test_loader = DataLoader(title_test_data, batch_size=batch_size)


In [ ]:
# using TensorDataset to transform the arrays into tensors for my overview textual data
overview_train_data = TensorDataset(torch.from_numpy(X_overview_train), torch.from_numpy(train_labels))
overview_val_data = TensorDataset(torch.from_numpy(X_overview_val), torch.from_numpy(val_labels))
overview_test_data = TensorDataset(torch.from_numpy(X_overview_test), torch.from_numpy(test_labels))

overview_train_loader = DataLoader(overview_train_data, batch_size=batch_size)
overview_val_loader = DataLoader(overview_val_data, batch_size=batch_size)
overview_test_loader = DataLoader(overview_test_data, batch_size=batch_size)

In [ ]:
# using TensorDataset to transform the arrays into tensors for my overview textual data
company_train_data = TensorDataset(torch.from_numpy(X_company_train), torch.from_numpy(train_labels))
company_val_data = TensorDataset(torch.from_numpy(X_company_val), torch.from_numpy(val_labels))
company_test_data = TensorDataset(torch.from_numpy(X_company_test), torch.from_numpy(test_labels))

company_train_loader = DataLoader(company_train_data, batch_size=batch_size)
company_val_loader = DataLoader(company_val_data, batch_size=batch_size)
company_test_loader = DataLoader(company_test_data, batch_size=batch_size)

In [ ]:
# Again, using TensorDataset to transform the arrays into tensors for my textual data.

# train dataset
three_feats_train_data = TensorDataset(torch.from_numpy(X_overview_train), torch.from_numpy(X_title_train), torch.from_numpy(train_labels))
# validation dataset
three_feats_val_data = TensorDataset(torch.from_numpy(X_overview_val), torch.from_numpy(X_title_val), torch.from_numpy(val_labels))
# test dataset
three_feats_test_data = TensorDataset(torch.from_numpy(X_overview_test),torch.from_numpy(X_title_test), torch.from_numpy(test_labels))

# dataloader for the model that uses three features as input
three_feats_train_loader = DataLoader(three_feats_train_data, batch_size=batch_size)
three_feats_val_loader = DataLoader(three_feats_val_data, batch_size=batch_size)
three_feats_test_loader = DataLoader(three_feats_test_data, batch_size=batch_size)


In [ ]:
# train dataset
four_feats_train_data = TensorDataset(torch.from_numpy(X_overview_train), torch.from_numpy(X_title_train), torch.from_numpy(X_company_train),torch.from_numpy(train_labels))
# validation dataset
four_feats_val_data = TensorDataset(torch.from_numpy(X_overview_val), torch.from_numpy(X_title_val),torch.from_numpy(X_company_val), torch.from_numpy(val_labels))
# test dataset
four_feats_test_data = TensorDataset(torch.from_numpy(X_overview_test),torch.from_numpy(X_title_test), torch.from_numpy(X_company_test),torch.from_numpy(test_labels))

# Dataloader
four_feats_train_loader = DataLoader(four_feats_train_data, batch_size=batch_size)
four_feats_val_loader = DataLoader(four_feats_val_data, batch_size=batch_size)
four_feats_test_loader = DataLoader(four_feats_test_data, batch_size=batch_size)


## Hyperparameters

In [ ]:
# Hyperparameters
batch_size = 150
epochs = 20
clip = 5
vocab_size = len(word_index)+1
output_size = train_labels.shape[1]
embedding_dim = 300
hidden_dim = 64
n_layers = 2
lr=0.001
criterion = nn.BCELoss()

## Model that takes 2 features as input

In [ ]:
# Building model for movie genre classification using textual and image features.
# CNN_LSTM2sc takes two features as input, The multimodal features can be either concatenated or summed. Change self.concat to False to sum.
class CNN_LSTM2sc(nn.Module):
    def __init__(self, vocab_size, weights_matrix, n_hidden, n_layers, n_out, concat=True):
        super(CNN_LSTM2sc, self).__init__()

        # LSTM for the text
        self.vocab_size, self.n_hidden, self.n_out, self.n_layers = vocab_size, n_hidden, n_out, n_layers
        num_embeddings, embedding_dim = weights_matrix.shape[0], weights_matrix.shape[1]
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.emb.weight.data.copy_(torch.from_numpy(weights_matrix))
        self.emb.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, self.n_hidden, self.n_layers, dropout=0.2, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.lstm_fc = nn.Linear(self.n_hidden, 128)
        self.concat = concat
        
        # CNN for the posters
        self.conv1 = nn.Conv2d(3, 32, 3) # Gives me depth of input.
        # pooling layers to reduce the image size
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.max_pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.max_pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(128, 128, 3)
        self.max_pool4 = nn.MaxPool2d(2)
        # to drop the nodes that are below the wanted weights
        self.cnn_dropout = nn.Dropout(0.1)
        if self.concat == True:
            self.cnn_fc = nn.Linear(4*4*128, 512) # Gives me number of features of input.         
            self.combined_fc1 = nn.Linear(640, 256) # Give me features of input.
            self.combined_fc2 = nn.Linear(256, 128)
            self.output_fc = nn.Linear(128, n_out)
        
        else:            
            self.cnn_fc = nn.Linear(4*4*128, 128) # when we sum, we want our textual and visual features to be of the same size so we can stack them
            # Concat layer for the combined feature space
            self.combined_fc1 = nn.Linear(128, 64) # Give me features of input.
            self.combined_fc2 = nn.Linear(64, 32)
            self.output_fc = nn.Linear(32, n_out)


    def forward(self, lstm_inp, cnn_inp):
        batch_size = lstm_inp.size(0)
        hidden = self.init_hidden(batch_size)
        lstm_inp = lstm_inp.long()
        embeds = self.emb(lstm_inp)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = self.dropout(lstm_out[:, -1])
        lstm_out = F.relu(self.lstm_fc(lstm_out))
        
        x = F.relu(self.conv1(cnn_inp))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)
        # that is how I get the correct dimensions for the CNN output that is then passed as input to the linear layer
        # need the input to be two dimensional again
        x = x.view(-1, 4*4*128)
        x = self.cnn_dropout(x)
        
        if self.concat == True:
            cnn_out = F.relu(self.cnn_fc(x))            
            #concatenating the cnn output and the lstm output
            combined_inp = torch.cat((cnn_out, lstm_out), 1)
            # activation function
            x_comb = F.relu(self.combined_fc1(combined_inp))
            x_comb = F.relu(self.combined_fc2(x_comb))
            # sigmoid to get probabilities for each label
            out = torch.sigmoid(self.output_fc(x_comb))
        else: # sum happens here
            cnn_out = F.relu(self.cnn_fc(x))            
            combined_inp = torch.stack([cnn_out, lstm_out], 1)            
            sum_comb = torch.sum(combined_inp,dim=1)           
            # activation function
            x_comb = F.relu(self.combined_fc1(sum_comb))
            x_comb = F.relu(self.combined_fc2(x_comb))
            # sigmoid to get probabilities for each label
            out = torch.sigmoid(self.output_fc(x_comb))     
                              
        return out

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                          weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        return hidden

## Model that takes 3 features as input

In [ ]:
# Building model for movie genre classification using textual and image features.
# CNN_LSTM3sc takes three features as input. The multimodal features can be either concatenated or summed. Change self.concat to False to sum.
class CNN_LSTM3sc(nn.Module):
    def __init__(self, vocab_size, weights_matrix, n_hidden, n_layers, n_out, concat=True):
        super(CNN_LSTM3sc, self).__init__()

        # LSTM for the text
        self.vocab_size, self.n_hidden, self.n_out, self.n_layers = vocab_size, n_hidden, n_out, n_layers
        num_embeddings, embedding_dim = weights_matrix.shape[0], weights_matrix.shape[1]
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.emb.weight.data.copy_(torch.from_numpy(weights_matrix))
        self.emb.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, self.n_hidden, self.n_layers, dropout=0.2, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.lstm_fc = nn.Linear(self.n_hidden, 128)
        self.concat = concat
        
        # CNN for the posters
        self.conv1 = nn.Conv2d(3, 32, 3) # Gives me depth of input.
        # pooling layers to reduce the image size
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.max_pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.max_pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(128, 128, 3)
        self.max_pool4 = nn.MaxPool2d(2)
        # to drop the nodes that are below the wanted weights
        self.cnn_dropout = nn.Dropout(0.1)
        if self.concat == True:
            self.cnn_fc = nn.Linear(4*4*128, 512) # Gives me number of features of input.
        # Concat layer for the combined feature space
            self.combined_fc1 = nn.Linear(640, 256) # Give me features of input.
            self.combined_fc2 = nn.Linear(256, 128)
            self.output_fc = nn.Linear(128, n_out)        
        else:            
            self.cnn_fc = nn.Linear(4*4*128, 128) # when we sum, we want our textual and visual features to be of the same size so we can stack them
            # Concat layer for the combined feature space
            self.combined_fc1 = nn.Linear(128, 64) # Give me features of input.
            self.combined_fc2 = nn.Linear(64, 32)
            self.output_fc = nn.Linear(32, n_out)

    def forward(self, lstm_in, cnn_inp, title_inp):
        batch_size = lstm_in.size(0)
        hidden = self.init_hidden(batch_size)
        lstm_in = lstm_in.long()
        title_inp = title_inp.long()
        embeds_lstm = self.emb(lstm_in)
        embeds_title = self.emb(title_inp)
        embeds = torch.cat((embeds_lstm,embeds_title),dim=1)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = self.dropout(lstm_out[:, -1])
        lstm_out = F.relu(self.lstm_fc(lstm_out))
       
        x = F.relu(self.conv1(cnn_inp))
        x = self.max_pool1(x)
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)
        # need the input to be two dimensional again
        x = x.view(-1, 4*4*128)        
        x = self.cnn_dropout(x)        
        if self.concat == True: # concatenate the multimodal features
            cnn_out = F.relu(self.cnn_fc(x))            
            #concatenating the cnn output and the lstm output
            combined_inp = torch.cat((cnn_out, lstm_out), 1)
            # activation function
            x_comb = F.relu(self.combined_fc1(combined_inp))
            x_comb = F.relu(self.combined_fc2(x_comb))
            # sigmoid to get probabilities for each label
            out = torch.sigmoid(self.output_fc(x_comb))
        else: # sum the multimodal features
            cnn_out = F.relu(self.cnn_fc(x))
            # sum happens here
            combined_inp = torch.stack([cnn_out, lstm_out], 1)            
            sum_comb = torch.sum(combined_inp,dim=1)
            # activation function
            x_comb = F.relu(self.combined_fc1(sum_comb))
            x_comb = F.relu(self.combined_fc2(x_comb))
            # # sigmoid to get probabilities for each label
            out = torch.sigmoid(self.output_fc(x_comb))     
                              
        return out

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                          weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        return hidden

## Model that takes 4 features as input

In [ ]:
# Building model for movie genre classification using textual and image features.
# CNN_LSTM4sc takes four features as input. The multimodal features can be either concatenated or summed. Change self.concat to False to sum.
class CNN_LSTM4sc(nn.Module):
    def __init__(self, vocab_size, weights_matrix, n_hidden, n_layers, n_out, concat=True):
        super(CNN_LSTM4sc, self).__init__()

        # LSTM for the text
        self.vocab_size, self.n_hidden, self.n_out, self.n_layers = vocab_size, n_hidden, n_out, n_layers
        num_embeddings, embedding_dim = weights_matrix.shape[0], weights_matrix.shape[1]
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.emb.weight.data.copy_(torch.from_numpy(weights_matrix))
        self.emb.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, self.n_hidden, self.n_layers, dropout=0.2, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.lstm_fc = nn.Linear(self.n_hidden, 128)
        self.concat = concat
        
        # CNN for the posters
        self.conv1 = nn.Conv2d(3, 32, 3) # Gives me depth of input.
        # pooling layers to reduce the image size
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.max_pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.max_pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(128, 128, 3)
        self.max_pool4 = nn.MaxPool2d(2)
        # to drop the nodes that are below the wanted weights
        self.cnn_dropout = nn.Dropout(0.1)
        if self.concat == True: # concatenate the multimodal features
            self.cnn_fc = nn.Linear(4*4*128, 512) # Gives me number of features of input.  
            # Concat layer for the combined feature space
            self.combined_fc1 = nn.Linear(640, 256) # Give me features of input.
            self.combined_fc2 = nn.Linear(256, 128)
            self.output_fc = nn.Linear(128, n_out)        
        else:   # sum the multimodal features
            self.cnn_fc = nn.Linear(4*4*128, 128) # when we sum, we want our textual and visual features to be of the same size so we can stack them
            # Concat layer for the combined feature space
            self.combined_fc1 = nn.Linear(128, 64) # Give me features of input.
            self.combined_fc2 = nn.Linear(64, 32)
            self.output_fc = nn.Linear(32, n_out)


    def forward(self, lstm_in, cnn_inp, title_inp, company_inp):
        batch_size = lstm_in.size(0)
        hidden = self.init_hidden(batch_size)
        lstm_in, title_inp,company_inp = lstm_in.long(), title_inp.long(), company_inp.long()
        embeds_lstm = self.emb(lstm_in)
        embeds_title = self.emb(title_inp)
        embeds_company = self.emb(company_inp)
        embeds = torch.cat((embeds_lstm,embeds_title,embeds_company),dim=1)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = self.dropout(lstm_out[:, -1])
        lstm_out = F.relu(self.lstm_fc(lstm_out))

        x = F.relu(self.conv1(cnn_inp))
        x = self.max_pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        
        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)
        # that is how I get the correct dimensions for the CNN output that is then passed as input to the linear layer
        # need the input to be two dimensional again
        x = x.view(-1, 4*4*128)
        
        x = self.cnn_dropout(x)
        if self.concat == True: # concatenate the multimodal features
            cnn_out = F.relu(self.cnn_fc(x))            
            #concatenating the cnn output and the lstm output
            combined_inp = torch.cat((cnn_out, lstm_out), 1)
            # activation function
            x_comb = F.relu(self.combined_fc1(combined_inp))
            x_comb = F.relu(self.combined_fc2(x_comb))
            # sigmoid to get probabilities for each label
            out = torch.sigmoid(self.output_fc(x_comb))
        else: # sum the multimodal features
            cnn_out = F.relu(self.cnn_fc(x))
            combined_inp = torch.stack([cnn_out, lstm_out], 1)            
            sum_comb = torch.sum(combined_inp,dim=1)
            # activation function
            x_comb = F.relu(self.combined_fc1(sum_comb))
            x_comb = F.relu(self.combined_fc2(x_comb))
            # sigmoid to get 1 and 0 on the one-hot-encoding formatted label
            out = torch.sigmoid(self.output_fc(x_comb))     
                              
        return out

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                          weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        return hidden

In [ ]:
# num_feats can be either 2,3 or 4 – i.e. the number of different features (overview, title, companies, poster) used as input to the model.
# model_num is an integer (0-9) to save the models we train.
def train_val(model,epochs, t_train_loader, image_train_loader, criterion, t_val_loader, image_val_loader, num_feats, model_num):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    model.train()
    for i in range(epochs):
        total_acc_train = 0
        total_loss_train = 0        
        for lstm, cnn in zip(t_train_loader, image_train_loader):
            cnn_inp, cnn_labels = cnn
            cnn_inp = cnn_inp.float()
            cnn_inp, cnn_labels = cnn_inp.to(device), cnn_labels.to(device)
            if num_feats == 2:
                lstm_inp,lstm_labels = lstm
                lstm_inp = lstm_inp.float()                
                lstm_inp, lstm_labels = lstm_inp.to(device), lstm_labels.to(device)                
                model.zero_grad()
                output = model(lstm_inp, cnn_inp)                
            elif num_feats == 3:
                lstm_inp, title_inp, lstm_labels = lstm        
                title_inp = title_inp.float()
                lstm_inp = lstm_inp.float()               
                lstm_inp = lstm_inp.to(device)
                title_inp = title_inp.to(device)
                lstm_labels =  lstm_labels.to(device)      
                model.zero_grad()
                output = model(lstm_inp, cnn_inp, title_inp)                
            else:
                lstm_inp, title_inp, company_inp, lstm_labels = lstm
                lstm_inp = lstm_inp.float()
                title_inp = title_inp.float()
                company_inp = company_inp.float()                
                lstm_inp, title_inp, company_inp,lstm_labels = lstm_inp.to(device),title_inp.to(device), company_inp.to(device), lstm_labels.to(device)
                
                model.zero_grad()
                output = model(lstm_inp, cnn_inp, title_inp,company_inp)
                
            loss = criterion(output.squeeze(), lstm_labels.float())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
        
            with torch.no_grad():
                acc = torch.abs(output.squeeze() - lstm_labels.float()).view(-1)
                acc = (1. - acc.sum() / acc.size()[0])
                total_acc_train += acc
                total_loss_train += loss.item()
      
        train_acc = total_acc_train/len(t_train_loader)
        train_loss = total_loss_train/len(t_train_loader)
        print('done training',train_acc)
        model.eval()
        total_acc_val = 0
        total_loss_val = 0
        with torch.no_grad():
            for lstm, cnn in zip(t_val_loader, image_val_loader):
                cnn_inp, cnn_labels = cnn
                cnn_inp = cnn_inp.float()
                cnn_inp, cnn_labels = cnn_inp.to(device), cnn_labels.to(device)
                if num_feats == 2:
                    lstm_inp,lstm_labels = lstm
                    lstm_inp = lstm_inp.float()                
                    lstm_inp, lstm_labels = lstm_inp.to(device), lstm_labels.to(device)                
                    model.zero_grad()
                    output = model(lstm_inp, cnn_inp) 
                elif num_feats == 3:
                    lstm_inp, title_inp, lstm_labels = lstm        
                    title_inp = title_inp.float()
                    lstm_inp = lstm_inp.float()               
                    lstm_inp = lstm_inp.to(device)
                    title_inp = title_inp.to(device)
                    lstm_labels =  lstm_labels.to(device)      
                    model.zero_grad()
                    output = model(lstm_inp, cnn_inp,title_inp)
                else:
                    lstm_inp, title_inp, company_inp, lstm_labels = lstm
                    lstm_inp = lstm_inp.float()
                    title_inp = title_inp.float()
                    company_inp = company_inp.float()                
                    lstm_inp, title_inp, company_inp,lstm_labels = lstm_inp.to(device),title_inp.to(device), company_inp.to(device), lstm_labels.to(device)
                    model.zero_grad()
                    output = model(lstm_inp, cnn_inp, title_inp,company_inp)
                    
                    
                val_loss = criterion(output.squeeze(), lstm_labels.float())
                acc = torch.abs(output.squeeze() - lstm_labels.float()).view(-1)
                acc = (1. - acc.sum() / acc.size()[0])
                total_acc_val += acc
                total_loss_val += val_loss.item()
        print("Saving model...") 
        # 10 diff models to save
        if model_num == 0: # concatenated title and poster
            torch.save(model.state_dict(), '../data/models/model0.pt')
        elif model_num == 1: # concatenated overview and poster
            torch.save(model.state_dict(), '../data/models/model1.pt')    
        elif model_num == 2: # concatenated overview, title and poster
            torch.save(model.state_dict(), '../data/models/model2.pt')
        elif model_num == 3: # concatenated companies and poster
            torch.save(model.state_dict(), '../data/models/model3.pt')
        elif model_num == 4: # concatenated overview, title, companies and poster
            torch.save(model.state_dict(), '../data/models/model4.pt')
        elif model_num == 5: # summed title and poster
            torch.save(model.state_dict(), '../data/models/model5.pt')
        elif model_num == 6: # summed overview and poster
            torch.save(model.state_dict(), '../data/models/model6.pt')
        elif model_num == 7: # summed overview, title and poster
            torch.save(model.state_dict(), '../data/models/model7.pt')
        elif model_num == 8: # summed companies and poster
            torch.save(model.state_dict(), '../data/models/model8.pt')
        elif model_num == 9: # summed overview, title, companies and poster
            torch.save(model.state_dict(), '../data/models/model9.pt')
    
        val_acc = total_acc_val/len(t_val_loader)
        val_loss = total_loss_val/len(t_val_loader)
        print(f'Epoch {i+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
        model.train()

In [ ]:
# This function returns accuracy and roc auc score of a model on the test set.
def test_acc_auc(model,text_test_loader, image_test_loader, criterion,num_feats):
    model.eval()
    correct = 0
    total = 0
    total_acc_test = 0
    total_loss_test = 0
    outputs = []
    with torch.no_grad():
        for lstm, cnn in zip(text_test_loader, image_test_loader):
            cnn_inp, cnn_labels = cnn
            cnn_inp = cnn_inp.float()
            cnn_inp, cnn_labels = cnn_inp.to(device), cnn_labels.to(device)
            if num_feats == 2:
                lstm_inp, lstm_labels = lstm
                lstm_inp = lstm_inp.float()
                
                lstm_inp, lstm_labels = lstm_inp.to(device), lstm_labels.to(device)
                
                out = model(lstm_inp, cnn_inp)
            elif num_feats == 3:
                lstm_inp, title_inp, lstm_labels = lstm
                lstm_inp = lstm_inp.float()
                title_inp = title_inp.float()
                lstm_inp, title_inp,lstm_labels = lstm_inp.to(device),title_inp.to(device), lstm_labels.to(device)
                out = model(lstm_inp, cnn_inp,title_inp)
            else:
                lstm_inp, title_inp, company_inp, lstm_labels = lstm
                lstm_inp = lstm_inp.float()
                title_inp = title_inp.float()
                company_inp = company_inp.float()                
                lstm_inp, title_inp, company_inp, lstm_labels = lstm_inp.to(device), title_inp.to(device), company_inp.to(device),  lstm_labels.to(device)                
                out = model(lstm_inp, cnn_inp, title_inp, company_inp)
                
            
                  
            outputs += list(out.cpu().data.numpy())
            loss = criterion(out.squeeze(), lstm_labels.float())
            acc = torch.abs(out.squeeze() - lstm_labels.float()).view(-1)
            acc = (1. - acc.sum() / acc.size()[0])
            total_acc_test += acc
            total_loss_test += loss.item()
    
    acc_test = total_acc_test/len(title_test_loader)
    loss_test = total_loss_test/len(title_test_loader)        
    
    np_out = np.array(outputs)
    y_pred = np.zeros(np_out.shape)
    y_pred[np_out>0.5]= 1 # threshold to assign a label is a probability of 0.5.
    y_pred = np.array(y_pred)    
    preds = np.transpose(y_pred)
    labels = np.transpose(test_labels)
    roc_auc = roc_auc_score(labels, preds) # calculating ROC-AUC score
    print(f'acc: {acc_test:.4f}, loss: {loss_test:.4f}, roc auc: {roc_auc}')
    return acc_test, roc_auc

## Title and image as input

In [ ]:
# concatenated title and poster input to model
model_title = CNN_LSTM2sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size)
model_title.to(device)
train_val(model_title,epochs, title_train_loader, img_train_loader,criterion, title_val_loader, img_val_loader, 2,0)
test_acc_auc(model_title, title_test_loader, img_test_loader, criterion,2)

In [ ]:
test_acc_auc(model_title, title_test_loader, img_test_loader, criterion,2)

In [ ]:
# train and test model with summed title and poster as input
model_title_sum = CNN_LSTM2sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size, concat=False)
model_title_sum.to(device)
train_val(model_title_sum,epochs, title_train_loader, img_train_loader,criterion, title_val_loader, img_val_loader, 2, 5)
test_acc_auc(model_title_sum, title_test_loader, img_test_loader, criterion,2)

## Overview and image as input

In [ ]:
# concatenated features
model_overview = CNN_LSTM2sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size)
model_overview.to(device)
train_val(model_overview, epochs, overview_train_loader, img_train_loader,criterion, overview_val_loader, img_val_loader, 2, 1)
test_acc_auc(model_overview, overview_test_loader, img_test_loader, criterion,2)

In [ ]:
# summed features
model_overview_sum = CNN_LSTM2sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size, concat=False)
model_overview_sum.to(device)
train_val(model_overview_sum, epochs, overview_train_loader, img_train_loader,criterion, overview_val_loader, img_val_loader, 2, 4)
test_acc_auc(model_overview_sum, overview_test_loader, img_test_loader, criterion,2)

## Title, image and poster as input

In [ ]:
# concatenated
model3 = CNN_LSTM3sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size)
model3.to(device)
train_val(model3, epochs, three_feats_train_loader, img_train_loader,criterion, three_feats_val_loader, img_val_loader, 3, 2)
test_acc_auc(model3, three_feats_test_loader, img_test_loader, criterion,3)

In [ ]:
# summed features
model3_sum = CNN_LSTM3sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size, concat=False)
model3_sum.to(device)
train_val(model3_sum, epochs, three_feats_train_loader, img_train_loader,criterion, three_feats_val_loader, img_val_loader, 3, 7)
test_acc_auc(model3_sum, three_feats_test_loader, img_test_loader, criterion,3)


## Production companies and Image

In [ ]:
# concatenated features
model_comp = CNN_LSTM2(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size)
model_comp.to(device)
train_val(model_comp, epochs, company_train_loader, img_train_loader,criterion, company_val_loader, img_val_loader, 2, 3)
test_acc_auc(model_comp, company_test_loader, img_test_loader, criterion, 2)

In [ ]:
# summed features
model_comp_sum = CNN_LSTM2(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size, concat=False)
model_comp_sum.to(device)
train_val(model_comp_sum, epochs, company_train_loader, img_train_loader,criterion, company_val_loader, img_val_loader, 2, 8)
test_acc_auc(model_comp_sum, company_test_loader, img_test_loader, criterion, 2)

## Overview, title, production companies & poster as input

In [ ]:
model4 = CNN_LSTM4sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size)
model4.to(device)
train_val(model4, epochs, four_feats_train_loader, img_train_loader, criterion, four_feats_val_loader, img_val_loader, 4, 4)
test_acc_auc(model4, four_feats_test_loader, img_test_loader, criterion, 4)

In [ ]:
model4_sum = CNN_LSTM4sc(vocab_size, word2vec_matrix, hidden_dim, n_layers, output_size, concat=False)
model4_sum.to(device)
train_val(model4_sum, epochs, four_feats_train_loader, img_train_loader, criterion, four_feats_val_loader, img_val_loader, 4, 9)
test_acc_auc(model4_sum, four_feats_test_loader, img_test_loader, criterion, 4)

In [ ]:
#action, comedy, crime, drama the lowest, horror, romance, thriller
# cooccurence of genres plot
l = df.genres.values

c = [list(combinations(i,2)) for i in l]
a = list(itertools.chain.from_iterable((i, i[::-1]) for c_ in c for i in c_))
dft = pd.DataFrame(a)
aba=pd.pivot_table(dft, index=0, columns=1, aggfunc='size', fill_value=0)
sns.heatmap(aba,cmap="YlGn")